In [83]:
import os
import geopandas as gpd
from sqlalchemy import create_engine
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pickle

In [84]:
def inspect_directory_structure(base_path):
    """Inspeccionar y listar la estructura de directorios."""
    directory_structure = []
    for year_folder in os.listdir(base_path):
        year_path = os.path.join(base_path, year_folder)
        if os.path.isdir(year_path):
            for province_folder in os.listdir(year_path):
                province_path = os.path.join(year_path, province_folder)
                if os.path.isdir(province_path):
                    for city_folder in os.listdir(province_path):
                        city_path = os.path.join(province_path, city_folder)
                        if os.path.isdir(city_path):
                            directory_structure.append((year_folder, province_folder, city_folder))
    return directory_structure

In [85]:
def create_database_and_extension(dbname, user, password, host, port):
    """Crear una base de datos y la extensión PostGIS."""
    # Conectar a la base de datos 'postgres' para crear la nueva base de datos
    con = psycopg2.connect(dbname='postgres', user=user, password=password, host=host, port=port)
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)  # Establecer el nivel de aislamiento a AUTOCOMMIT
    with con.cursor() as cur:
        try:
            cur.execute(f"CREATE DATABASE {dbname};")
            print(f"Base de datos '{dbname}' creada exitosamente.")
        except psycopg2.errors.DuplicateDatabase:
            print(f"La base de datos '{dbname}' ya existe.")
    con.close()  # Cerrar la conexión

    # Conectar a la nueva base de datos para crear la extensión PostGIS
    con = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    with con.cursor() as cur:
        cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
    con.close()

In [86]:
def read_shapefile_with_encodings(filepath, encodings=('utf-8', 'ISO-8859-1')):
    """Intentar leer un shapefile con diferentes codificaciones."""
    for encoding in encodings:
        try:
            return gpd.read_file(filepath, encoding=encoding)
        except UnicodeDecodeError:
            continue
    raise ValueError(f"No se pudo leer el archivo {filepath} con las codificaciones proporcionadas.")

In [87]:
def save_progress(progress, filename='progress.pkl'):
    """Guardar el progreso en un archivo."""
    with open(filename, 'wb') as f:
        pickle.dump(progress, f)

In [88]:
def load_progress(filename='progress.pkl'):
    """Cargar el progreso desde un archivo."""
    if os.path.exists(filename) and os.path.getsize(filename) > 0:
        with open(filename, 'rb') as f:
            try:
                return pickle.load(f)
            except EOFError:
                print(f"El archivo {filename} está vacío o corrupto. Reiniciando progreso.")
                return None
    return None

In [ ]:
def main():
    # Solicita al usuario los datos mediante input
    base_path = input("Ingrese la ruta base (por ejemplo, C:\\Users\\jurintivet\\Downloads\\Parcelas): ")
    user = input("Ingrese el nombre de usuario de la base de datos (por defecto 'postgres'): ")
    password = input("Ingrese la contraseña de la base de datos: ")
    host = input("Ingrese el host (por defecto 'localhost'): ")
    port = input("Ingrese el puerto (por defecto '5432'): ")

    progress = load_progress()

    # Imprime los valores ingresados para verificar
    print(f"Ruta base: {base_path}")
    print(f"Usuario: {user}")
    print(f"Contraseña: {password}")
    print(f"Host: {host}")
    print(f"Puerto: {port}")

    progress = load_progress()

    # Inspeccionar la estructura de directorios
    directory_structure = inspect_directory_structure(base_path)
    print("Estructura de directorios encontrada:")
    for year, province, city in directory_structure:
        print(f"Año: {year}, Provincia: {province}, Ciudad: {city}")

    # Proceder con la creación de bases de datos y carga de shapefiles
    for year, province, city in directory_structure:
        progress_key = (year, province, city)
        if progress_key in progress["completed"]:
            continue

        province_path = os.path.join(base_path, year, province)
        city_path = os.path.join(province_path, city)

        # Usar el nombre de la provincia y año como nombre de la base de datos, reemplazando espacios por guiones bajos
        dbname = f"{province.lower().replace(' ', '_')}_{year}"
        create_database_and_extension(dbname, user, password, host, port)

        # Crear conexión a la base de datos de la provincia
        engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')
        
        # Buscar shapefiles en la carpeta de la ciudad
        for file in os.listdir(city_path):
            if file.endswith('.shp'):
                shapefile_path = os.path.join(city_path, file)
                table_name = f"{city.lower().replace(' ', '_')}_basic"
                try:
                    gdf = read_shapefile_with_encodings(shapefile_path)
                    gdf.to_postgis(table_name, engine, if_exists='replace')
                    print(f"Shapefile '{file}' cargado en la tabla '{table_name}' de la base de datos '{dbname}'.")
                except Exception as e:
                    print(f"No se pudo cargar el archivo {shapefile_path}. Error: {e}")

        # Marcar progreso
        progress["completed"].add(progress_key)
        save_progress(progress)

if __name__ == "__main__":
    main()